### Импорт нужных библиотек

In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModel
import faiss
model = SentenceTransformer("cointegrated/rubert-tiny2")
import numpy as np


/home/valera/ds_bootcamp/ds-phase-2/10-nlp/myenv_week8/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Считывание Датафрейма pd.read_csv("ПУТЬ К ДАТАФРЕЙМУ")
df = pd.read_csv("/home/valera/ds_bootcamp/ds-phase-2/11-nn-ext/progect/resources/movies.csv")

#### Основные функций обработки Датафрейма 
##### Учтите, что если формат данный в вашем датафрейме отличается, возможно нужно будет использовать другие конструкций обработки 
##### На ембединг обработка не повлияет, обработка нужна для корректной работы фильтров

In [100]:
#import re
#df["genres"] = df["genres"].apply(lambda x: re.sub(r'[\"\'{}\[\]]', '', x))

# df['genres'] = df['genres'].apply(lambda x: x.replace('[','').replace(']','').replace(' ','').replace("'","").split(','))

# #  изменение даты из типа строка в целочисленное 
# df['year'] = df['year'].apply(lambda x: int(x))

# # удаление описания мменьше 55 символов
# df = df[df["description"].str.len() >= 55]

#df['imdb'].fillna(0.0, inplace= True)
#df['genres'].fillna('Нет данных', inplace= True)
#df['director'].fillna('Нет данных', inplace= True)


# df.drop_duplicates(ignore_index= True , inplace= True)

#df.drop('embeddings',axis=1, inplace= True)

#df.loc[df["img_url"] == "/img/v2/nopicture/308x462@2x.png", "img_url"] = "https://upload.wikimedia.org/wikipedia/commons/a/a1/Out_Of_Poster.jpg"

#df['genres'] = df['genres'].apply(lambda x: 'Нет данных' if pd.isna(x) else x )

/tmp/ipykernel_27737/38251508.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['director'].fillna('Нет данных', inplace= True)


#### Сохранение обработаного датафрейма

In [79]:
#df.to_csv('/home/valera/ds_bootcamp/ds-phase-2/11-nn-ext/progect/movies_data2.csv', index = False)

/home/valera/Загрузки


### Создание индексов и ембедингов для поиска по синусному сходству

In [15]:
import numpy as np


# создаем вектора вложений с помощью модели SentenceTransformer
embeddings = model.encode(df["description"])

# # # создаем индекс Faiss 
index = faiss.IndexFlatIP(embeddings.shape[1])

# # # добавляем вектора вложений в индекс Faiss
index.add(np.array(df['embeddings'].tolist()))

### Создание индексов и ембедингов для поиска по синусному сходству с применение SVD

In [ ]:
# # С этим пораметром  можно поиграться и посмотреть как оно работает.  Мне кажется что около 35-10 лучше всего работает
# # Где то больше лучше, где-то меньше. На маленьких размерностях лучше ищет не очень часто встречающиеся термины.
# # Понижаем размерность эмбедингов с 312 компонентов до num_components 
num_components = 15
n=5
# Сингулярное разложение эмбедингов #####################
U, S, V = np.linalg.svd(embeddings, full_matrices=False)#
                                                        #
U = U[:, :num_components]                               #
S = np.diag(S[:num_components])                         #
V = V[:num_components, :]                               #
                                                        #
                                                        #
reduced_embeddings = np.dot(U, np.dot(S, V))            #
reduced_embeddings.shape                                #
#########################################################   

# Делаем новый индекс с разложенными эмбедингами
reduced_index = faiss.IndexFlatIP(reduced_embeddings.shape[1])
reduced_index.add(reduced_embeddings)

# Сингулярное разложение текста от юзера ##########################################
user_input = model.encode("Дружба")                                               #
user_input = user_input.reshape(1, -1)                                            #
                                                                                  #
U_reduced, S_reduced, Vt_reduced = np.linalg.svd(user_input, full_matrices=False) #
                                                                                  #
U_reduced = U_reduced[:, :num_components]                                         #
S_reduced = np.diag(S_reduced[:num_components])                                   #
V_reduced = Vt_reduced[:num_components, :]                                        #
                                                                                  #
reduced_user_input = np.dot(U_reduced, np.dot(S_reduced, V_reduced))              #
reduced_output = reduced_index.search(reduced_user_input, n)                      #
###################################################################################

# Смотрим какие описания нашли
for i in range(n):
    print(df.iloc[reduced_output[1][0][i]].loc['description'])

### Сохранение embeddings и загрузка

In [20]:
 #  np.save('/home/valera/ds_bootcamp/ds-phase-2/11-nn-ext/progect/test.npy', embeddings )
 #  np.load('resources/embeding.npy')

#### Получения уникальных данных для преобразования в лист 


In [31]:
# Получание списка, для использования в фильтре
lst_genre = []
for i in df['genres'].unique():
    for q in i.split(' '):
        if q in lst_genre:
            pass
        else:
            lst_genre.append(q)

In [33]:
lst_genre =['триллер,','боевик','драма,','комедия','мелодрама,','комедия,','детектив,','криминал','боевик,','фантастика,','фэнтези','детский,','семейный','фантастика', 
            'короткометражный','детектив','мелодрама','драма','триллер','мультфильмы,','приключения','короткометражный,','мультфильмы','приключения,',
            'семейный,','документальный','ужасы','биография','криминал,','документальный,','исторический,','аниме,','вестерн','ужасы,','спорт',
            'музыкальный','военный,','мюзикл','исторический','Нет','данных','биография,','вестерн,','мюзикл,','военный','музыкальный,','детский','эротика','фэнтези,','аниме']

In [ ]:
for i in df['actors'].iloc[:50]:
    for q in i.split(', '):
        print(q)

In [ ]:
lst_actor = []
for i in df['actors']:
    for q in i.split(', '):

        if q in lst_actor:
            pass
        else:
            lst_actor.append(q)
        
    

#### Для больших листов, можно использовать для сохранения в файл

In [ ]:
import pickle
with open('/home/valera/ds_bootcamp/ds-phase-2/11-nn-ext/progect/lst_actor.pkl', 'wb') as f:
    pickle.dump(lst_actor, f)
    
with open('/home/valera/ds_bootcamp/ds-phase-2/11-nn-ext/progect/lst_actor.pkl', 'rb') as f:
    lst_actor_loaded = pickle.load(f)

#### Функция для выдачи предсказаний [indices[0]]['movie_title'] заменив название столбца , можна заменить на вывод нужных данных, таких как описание или год фильма

In [5]:
  # Функция для поиска фильмов по сходству
def find_similar_movies(user_input, top_n=5):
    user_input = model.encode(user_input)
    user_input = user_input.reshape(1,-1)
    distances, indices = index.search(user_input, top_n)
    similar_movies = df.iloc[indices[0]]['movie_title']
    #return list(indices[0])
    for i in similar_movies:
        print(i)
        
find_similar_movies('фильм про собак')

Они были простыми солдатами
Большой секрет для маленькой компании
Дорога в погибель
Рита, Сью и Боб тоже
Свои не свои


#### Ниже функций фильтов и их проверки


In [65]:
#Фильтры
test = ['эротика']
target_set = set(test)
# фильтруем DataFrame
df[df['genres'].apply(lambda x: target_set.issubset(x.split(', ')))]


In [68]:
  # Функция для поиска фильмов по сходству
def find_similar_movies(user_input, top_n=5):
    user_input = model.encode(user_input)
    user_input = user_input.reshape(1,-1)
    distances, indices = index.search(user_input, top_n)
    similar_movies = df.iloc[indices[0]]['genres']
    #return list(indices[0])
    for i in similar_movies:
        print(i)
        
find_similar_movies('xktys')

эротика
эротика
эротика
триллер, эротика
эротика


In [11]:
min = 2013
max = 2014

In [47]:
df[(df['year'] >= min) & (df['year'] <= max)]

TypeError: '>=' not supported between instances of 'int' and 'builtin_function_or_method'

In [5]:
# Фильтр актеров
test = ['Арбен Байрактарай']
target_set = set(test)
# фильтруем DataFrame
df[df['actors'].apply(lambda x: target_set.issubset(x.split(', ')))]

NameError: name 'df' is not defined